In [1]:
import pandas as pd
import numpy as np
import scipy
pd.set_option("display.max_columns", 9999)
from point_partage import transformers
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import Imputer
from StringIO import StringIO
from hyperopt import hp, fmin, tpe
from hyperopt.base import Trials, STATUS_OK
from hyperopt.fmin import space_eval
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import cross_val_score

# Data loading

In [2]:
trivago_data_1 = pd.read_csv("input_data/trivago_raw_2017-03-01.csv", sep=";")
trivago_data_2 = pd.read_csv("input_data/trivago_raw_2017-03-02.csv", sep=";")

In [3]:
trivago_data_1.head()

,"﻿""Hotel Name""",POS,trivagoID,PartnerRef,trivago URL,City,Stars,Rating,Status,Clicks,Hotel Impr,Bookings,Avg Pos,Avg Hotel Pos,Avg CPC,Bid CPC,Cost,Opportunity CPC,ABV,Gross Rev,Net Rev,Net Rev/Click,Avg Min Price,Profit,ROAS,CPA,Region,Country,Top Pos Share,Invisible Ratio,Outbid Ratio,Booking Rate,Margin %,Last Pushed
0,Grand-Hotel Cap Ferrat,FR,96367,19958,http://www.trivago.de/?geo_distance_item=96367,Saint Jean-Cap Ferrat,5.0,89.0,No Change,68,1196,0,2.8,5.3,0.15,0.15,10.20,0.22,NaN,0.0,0.0,0.0,518.0,-10.20,0.0,NaN,Provence-Alpes-Côte d'Azur,France,0.1000,0.9,0.9,0.0000,NaN,NaN
1,Château de la Chèvre d'Or,FR,93388,174801,http://www.trivago.de/?geo_distance_item=93388,Éze,5.0,91.0,No Change,59,457,0,2.0,2.1,0.15,0.15,8.85,0.49,NaN,0.0,0.0,0.0,495.0,-8.85,0.0,NaN,Provence-Alpes-Côte d'Azur,France,0.1328,0.9,0.9,0.0000,NaN,NaN
2,Contact Hotel - Sable et Soleil,FR,1825077,5129701,http://www.trivago.de/?geo_distance_item=1825077,Fréjus,2.0,73.0,No Change,42,307,0,1.0,12.1,0.36,0.36,15.12,0.55,NaN,0.0,0.0,0.0,132.0,-15.12,0.0,NaN,Provence-Alpes-Côte d'Azur,France,0.9000,0.1,0.1,0.0000,NaN,NaN
3,Canary Heights,FR,4385630,11791260,http://www.trivago.de/?geo_distance_item=4385630,London,3.0,77.0,No Change,41,290,0,1.0,16.0,0.26,0.26,10.66,0.51,NaN,0.0,0.0,0.0,281.0,-10.66,0.0,NaN,England,United Kingdom,0.9000,0.1,0.1,0.0000,NaN,NaN
4,Carnac Thalasso & spa resort Les Salines,FR,86005,565541,http://www.trivago.de/?geo_distance_item=86005,Carnac,4.0,81.0,No Change,41,381,1,1.4,14.2,0.32,0.32,13.12,0.46,91.2,91.2,0.0,0.0,178.0,-13.12,0.0,13.12,Brittany,France,0.6780,0.1,0.1,0.0244,0.0,NaN


In [4]:
trivago_data_1["date"] = "2017-03-01"
trivago_data_2["date"] = "2017-03-02"

In [5]:
trivago_data = pd.concat([trivago_data_1, trivago_data_2], ignore_index=True)

In [6]:
trivago_data.head()

,"﻿""Hotel Name""",POS,trivagoID,PartnerRef,trivago URL,City,Stars,Rating,Status,Clicks,Hotel Impr,Bookings,Avg Pos,Avg Hotel Pos,Avg CPC,Bid CPC,Cost,Opportunity CPC,ABV,Gross Rev,Net Rev,Net Rev/Click,Avg Min Price,Profit,ROAS,CPA,Region,Country,Top Pos Share,Invisible Ratio,Outbid Ratio,Booking Rate,Margin %,Last Pushed,date
0,Grand-Hotel Cap Ferrat,FR,96367,19958,http://www.trivago.de/?geo_distance_item=96367,Saint Jean-Cap Ferrat,5.0,89.0,No Change,68,1196,0,2.8,5.3,0.15,0.15,10.20,0.22,NaN,0.0,0.0,0.0,518.0,-10.20,0.0,NaN,Provence-Alpes-Côte d'Azur,France,0.1000,0.9,0.9,0.0000,NaN,NaN,2017-03-01
1,Château de la Chèvre d'Or,FR,93388,174801,http://www.trivago.de/?geo_distance_item=93388,Éze,5.0,91.0,No Change,59,457,0,2.0,2.1,0.15,0.15,8.85,0.49,NaN,0.0,0.0,0.0,495.0,-8.85,0.0,NaN,Provence-Alpes-Côte d'Azur,France,0.1328,0.9,0.9,0.0000,NaN,NaN,2017-03-01
2,Contact Hotel - Sable et Soleil,FR,1825077,5129701,http://www.trivago.de/?geo_distance_item=1825077,Fréjus,2.0,73.0,No Change,42,307,0,1.0,12.1,0.36,0.36,15.12,0.55,NaN,0.0,0.0,0.0,132.0,-15.12,0.0,NaN,Provence-Alpes-Côte d'Azur,France,0.9000,0.1,0.1,0.0000,NaN,NaN,2017-03-01
3,Canary Heights,FR,4385630,11791260,http://www.trivago.de/?geo_distance_item=4385630,London,3.0,77.0,No Change,41,290,0,1.0,16.0,0.26,0.26,10.66,0.51,NaN,0.0,0.0,0.0,281.0,-10.66,0.0,NaN,England,United Kingdom,0.9000,0.1,0.1,0.0000,NaN,NaN,2017-03-01
4,Carnac Thalasso & spa resort Les Salines,FR,86005,565541,http://www.trivago.de/?geo_distance_item=86005,Carnac,4.0,81.0,No Change,41,381,1,1.4,14.2,0.32,0.32,13.12,0.46,91.2,91.2,0.0,0.0,178.0,-13.12,0.0,13.12,Brittany,France,0.6780,0.1,0.1,0.0244,0.0,NaN,2017-03-01


In [7]:
trivago_data.date.unique()

array(['2017-03-01', '2017-03-02'], dtype=object)

In [8]:
indexed_data = trivago_data.set_index(["PartnerRef","date"])

In [9]:
data_with_next_values = transformers.NextValueAdder(cols_to_process=["Clicks", "Bid CPC"]).fit_transform(indexed_data)

In [10]:
data_with_next_values.head()

﻿"Hotel Name" POS  trivagoID  \
PartnerRef date                                              
1          2017-03-01         Saint Georges  FR      47025   
           2017-03-02         Saint Georges  FR      47025   
9          2017-03-02        Hilton Anaheim  FR      14795   
19         2017-03-01  Plaza Hotel & Casino  FR     129571   
           2017-03-02  Plaza Hotel & Casino  FR     129571   

                                                           trivago URL  \
PartnerRef date                                                          
1          2017-03-01   http://www.trivago.de/?geo_distance_item=47025   
           2017-03-02   http://www.trivago.de/?geo_distance_item=47025   
9          2017-03-02   http://www.trivago.de/?geo_distance_item=14795   
19         2017-03-01  http://www.trivago.de/?geo_distance_item=129571   
           2017-03-02  http://www.trivago.de/?geo_distance_item=129571   

                            City  Stars  Rating     Status  Clicks  \
PartnerRef date                                                      
1          2017-03-01     London    4.0    75.0  No Change       0   
           2017-03-02     London    4.0    75.0  No Change       0   
9          2017-03-02    Anaheim    4.0    81.0  No Change       0   
19         2017-03-01  Las Vegas    3.0    73.0  No Change       0   
           2017-03-02  Las Vegas    3.0    73.0  No Change       0   

                       Hotel Impr  Bookings  Avg Pos  Avg Hotel Pos  Avg CPC  \
PartnerRef date                                                                
1          2017-03-01          15         0      3.0           75.1      NaN   
           2017-03-02          82         0      2.0           27.7      NaN   
9          2017-03-02           7         0      6.0            1.9      NaN   
19         2017-03-01         219         0      9.8           11.9      NaN   
           2017-03-02         187         0     11.0           12.3      NaN   

                       Bid CPC  Cost  Opportunity CPC  ABV  Gross Rev  \
PartnerRef date                                                         
1          2017-03-01     0.35   0.0             0.77  NaN        0.0   
           2017-03-02     0.35   0.0             0.77  NaN        0.0   
9          2017-03-02     0.15   0.0             0.30  NaN        0.0   
19         2017-03-01     0.03   0.0             0.36  NaN        0.0   
           2017-03-02     0.03   0.0             0.37  NaN        0.0   

                       Net Rev  Net Rev/Click  Avg Min Price  Profit  ROAS  \
PartnerRef date                                                              
1          2017-03-01      0.0            NaN          601.0     0.0   NaN   
           2017-03-02      0.0            NaN          308.0     0.0   NaN   
9          2017-03-02      0.0            NaN          110.0     0.0   NaN   
19         2017-03-01      0.0            NaN          152.0     0.0   NaN   
           2017-03-02      0.0            NaN          395.0     0.0   NaN   

                       CPA      Region         Country  Top Pos Share  \
PartnerRef date                                                         
1          2017-03-01  NaN     England  United Kingdom            0.1   
           2017-03-02  NaN     England  United Kingdom            0.1   
9          2017-03-02  NaN  California             USA            0.1   
19         2017-03-01  NaN      Nevada             USA            0.1   
           2017-03-02  NaN      Nevada             USA            0.1   

                       Invisible Ratio  Outbid Ratio  Booking Rate  Margin %  \
PartnerRef date                                                                
1          2017-03-01              0.1           NaN           NaN       NaN   
           2017-03-02              0.9           0.9           NaN       NaN   
9          2017-03-02              0.9           0.9           NaN       NaN   
19         2017-03-01              0.9           0.9 

In [11]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_data = data_with_next_values.select_dtypes(include=numerics)
numeric_data.head()


trivagoID  Stars  Rating  Clicks  Hotel Impr  Bookings  \
PartnerRef date                                                                 
1          2017-03-01      47025    4.0    75.0       0          15         0   
           2017-03-02      47025    4.0    75.0       0          82         0   
9          2017-03-02      14795    4.0    81.0       0           7         0   
19         2017-03-01     129571    3.0    73.0       0         219         0   
           2017-03-02     129571    3.0    73.0       0         187         0   

                       Avg Pos  Avg Hotel Pos  Avg CPC  Bid CPC  Cost  \
PartnerRef date                                                         
1          2017-03-01      3.0           75.1      NaN     0.35   0.0   
           2017-03-02      2.0           27.7      NaN     0.35   0.0   
9          2017-03-02      6.0            1.9      NaN     0.15   0.0   
19         2017-03-01      9.8           11.9      NaN     0.03   0.0   
           2017-03-02     11.0           12.3      NaN     0.03   0.0   

                       Opportunity CPC  ABV  Gross Rev  Net Rev  \
PartnerRef date                                                   
1          2017-03-01             0.77  NaN        0.0      0.0   
           2017-03-02             0.77  NaN        0.0      0.0   
9          2017-03-02             0.30  NaN        0.0      0.0   
19         2017-03-01             0.36  NaN        0.0      0.0   
           2017-03-02             0.37  NaN        0.0      0.0   

                       Net Rev/Click  Avg Min Price  Profit  ROAS  CPA  \
PartnerRef date                                                          
1          2017-03-01            NaN          601.0     0.0   NaN  NaN   
           2017-03-02            NaN          308.0     0.0   NaN  NaN   
9          2017-03-02            NaN          110.0     0.0   NaN  NaN   
19         2017-03-01            NaN          152.0     0.0   NaN  NaN   
           2017-03-02            NaN          395.0     0.0   NaN  NaN   

                       Top Pos Share  Invisible Ratio  Outbid Ratio  \
PartnerRef date                                                       
1          2017-03-01            0.1              0.1           NaN   
           2017-03-02            0.1              0.9           0.9   
9          2017-03-02            0.1              0.9           0.9   
19         2017-03-01            0.1              0.9           0.9   
           2017-03-02            0.1              0.9           0.9   

                       Booking Rate  Margin %  Last Pushed  next_Clicks  \
PartnerRef date                                                           
1          2017-03-01           NaN       NaN          NaN          0.0   
           2017-03-02           NaN       NaN          NaN          NaN   
9          2017-03-02           NaN       NaN          NaN          NaN   
19         2017-03-01           NaN       NaN          NaN          0.0   
           2017-03-02           NaN       NaN          NaN          NaN   

                       next_Bid CPC  
PartnerRef date                      
1          2017-03-01          0.35  
           2017-03-02           NaN  
9          2017-03-02           NaN  
19         2017-03-01          0.03  
           2017-03-02           NaN

In [12]:
numeric_data.isnull().sum()/float(len(numeric_data))

trivagoID          0.000000
Stars              0.000021
Rating             0.020737
Clicks             0.000000
Hotel Impr         0.000000
Bookings           0.000000
Avg Pos            0.646864
Avg Hotel Pos      0.023685
Avg CPC            0.958210
Bid CPC            0.000021
Cost               0.000000
Opportunity CPC    0.041674
ABV                0.999080
Gross Rev          0.000000
Net Rev            0.000000
Net Rev/Click      0.958210
Avg Min Price      0.023189
Profit             0.000000
ROAS               0.958210
CPA                0.999080
Top Pos Share      0.646684
Invisible Ratio    0.026983
Outbid Ratio       0.070560
Booking Rate       0.958210
Margin %           0.999080
Last Pushed        1.000000
next_Clicks        0.639021
next_Bid CPC       0.639032
dtype: float64

In [13]:
target = 'next_Clicks'
numeric_data.loc[numeric_data[target].isnull(), target] = 0

/home/groupevsc.com/victor_landeau/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
y = numeric_data['next_Clicks']
x_numeric = numeric_data.drop(['next_Clicks', "Last Pushed"], axis=1)

In [15]:
x_numeric.columns

Index([u'trivagoID', u'Stars', u'Rating', u'Clicks', u'Hotel Impr',
       u'Bookings', u'Avg Pos', u'Avg Hotel Pos', u'Avg CPC', u'Bid CPC',
       u'Cost', u'Opportunity CPC', u'ABV', u'Gross Rev', u'Net Rev',
       u'Net Rev/Click', u'Avg Min Price', u'Profit', u'ROAS', u'CPA',
       u'Top Pos Share', u'Invisible Ratio', u'Outbid Ratio', u'Booking Rate',
       u'Margin %', u'next_Bid CPC'],
      dtype='object')

In [16]:
x_numeric.shape

(94615, 26)

In [17]:
imputed_data = Imputer(strategy='median', axis=1).fit_transform(x_numeric)

In [18]:
vt = VarianceThreshold(threshold=0.02)
x = vt.fit_transform(imputed_data)

In [19]:
list_features = list(x_numeric.columns[vt.get_support()]) 
removed_features = list(x_numeric.columns[~vt.get_support()])
print "Kept columns are: {}".format(list_features)
print "Removed columns are: {}".format(str(removed_features))


Kept columns are: ['trivagoID', 'Stars', 'Rating', 'Clicks', 'Hotel Impr', 'Avg Pos', 'Avg Hotel Pos', 'Avg CPC', 'Cost', 'Opportunity CPC', 'ABV', 'Gross Rev', 'Net Rev/Click', 'Avg Min Price', 'Profit', 'ROAS', 'CPA', 'Top Pos Share', 'Invisible Ratio', 'Outbid Ratio', 'Booking Rate', 'Margin %', 'next_Bid CPC']
Removed columns are: ['Bookings', 'Bid CPC', 'Net Rev']


In [20]:
import time

start_time = time.time()
rf = RandomForestRegressor(n_estimators=50, n_jobs=4)
rf.fit(x, y)

print time.time() - start_time

8.30675411224


In [21]:
importances = pd.DataFrame({"features": list_features,
                           "importances": rf.feature_importances_})
importances.sort("importances", ascending=False)

/home/groupevsc.com/victor_landeau/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  This is separate from the ipykernel package so we can avoid doing imports until


,features,importances
3,Clicks,0.201881
14,Profit,0.139182
8,Cost,0.115114
4,Hotel Impr,0.082993
22,next_Bid CPC,0.077754
13,Avg Min Price,0.056121
0,trivagoID,0.051781
6,Avg Hotel Pos,0.049476
2,Rating,0.033178
16,CPA,0.030876


# Pipeline

In [22]:
x_numeric.isnull().sum()/float(len(x_numeric))

trivagoID          0.000000
Stars              0.000021
Rating             0.020737
Clicks             0.000000
Hotel Impr         0.000000
Bookings           0.000000
Avg Pos            0.646864
Avg Hotel Pos      0.023685
Avg CPC            0.958210
Bid CPC            0.000021
Cost               0.000000
Opportunity CPC    0.041674
ABV                0.999080
Gross Rev          0.000000
Net Rev            0.000000
Net Rev/Click      0.958210
Avg Min Price      0.023189
Profit             0.000000
ROAS               0.958210
CPA                0.999080
Top Pos Share      0.646684
Invisible Ratio    0.026983
Outbid Ratio       0.070560
Booking Rate       0.958210
Margin %           0.999080
next_Bid CPC       0.639032
dtype: float64

In [23]:
np.isnan(x_numeric.as_matrix()).sum()

848652

In [24]:
imputed_data2 = transformers.NumericalFeatureImputation().fit_transform(x_numeric)

trivagoID          0
Stars              0
Rating             0
Clicks             0
Hotel Impr         0
Bookings           0
Avg Pos            0
Avg Hotel Pos      0
Avg CPC            0
Bid CPC            0
Cost               0
Opportunity CPC    0
ABV                0
Gross Rev          0
Net Rev            0
Net Rev/Click      0
Avg Min Price      0
Profit             0
ROAS               0
CPA                0
Top Pos Share      0
Invisible Ratio    0
Outbid Ratio       0
Booking Rate       0
Margin %           0
next_Bid CPC       0
dtype: int64


In [25]:
np.isnan(imputed_data).sum()

0

In [26]:
np.isnan(imputed_data2).sum()

trivagoID          0
Stars              0
Rating             0
Clicks             0
Hotel Impr         0
Bookings           0
Avg Pos            0
Avg Hotel Pos      0
Avg CPC            0
Bid CPC            0
Cost               0
Opportunity CPC    0
ABV                0
Gross Rev          0
Net Rev            0
Net Rev/Click      0
Avg Min Price      0
Profit             0
ROAS               0
CPA                0
Top Pos Share      0
Invisible Ratio    0
Outbid Ratio       0
Booking Rate       0
Margin %           0
next_Bid CPC       0
dtype: int64

In [75]:
processing_pipeline = Pipeline({
        ("imputation", transformers.NumericalFeatureImputation(strategy="mean")),
        ("ConvertToNp", transformers.PandasDfToNpArrayConverter()),
    })
model_pipeline = Pipeline({
        ("variance_threshold", VarianceThreshold()),
        ("random_forest", RandomForestRegressor(n_jobs=4))
})

params = {
    "variance_threshold__threshold": 0.02,
    "random_forest__n_estimators": 10
}

model_pipeline.set_params(**params)
x = processing_pipeline.fit_transform(x_numeric)
y_pred = model_pipeline.fit_transform(x, y)

trivagoID          0
Stars              0
Rating             0
Clicks             0
Hotel Impr         0
Bookings           0
Avg Pos            0
Avg Hotel Pos      0
Avg CPC            0
Bid CPC            0
Cost               0
Opportunity CPC    0
ABV                0
Gross Rev          0
Net Rev            0
Net Rev/Click      0
Avg Min Price      0
Profit             0
ROAS               0
CPA                0
Top Pos Share      0
Invisible Ratio    0
Outbid Ratio       0
Booking Rate       0
Margin %           0
next_Bid CPC       0
dtype: int64


/home/groupevsc.com/victor_landeau/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [85]:
processing_pipeline.steps[0]

('imputation', NumericalFeatureImputation(strategy='mean'))

# Hyperopt

In [80]:
parameters_space = {
    #"imputation__strategy": hp.choice("imputation_strategy", ["mean", "median"]),
    "variance_threshold__threshold": hp.uniform("variance_threshold", 0, 1),
    'random_forest__max_depth': hp.choice('max_depth', list(range(2, 10)) + [None]),
    'random_forest__max_features': hp.choice('max_features', [None, 'sqrt', 'log2']),
    'random_forest__n_estimators': hp.quniform('n_estimators', 50, 800, 1),
    'random_forest__criterion': hp.choice('criterion', ["gini", "entropy"]),
    'random_forest__bootstrap': hp.choice('bootstrap', [False, True])
}

In [81]:
def get_best_params(estimator_to_optimize, space, cv, x_train, y_train, max_evals):
    objective = get_objective_function(estimator_to_optimize, cv, x_train, y_train)
    trials = Trials()
    best_space_params = fmin(fn=objective,
                             space=space,
                             algo=tpe.suggest,
                             max_evals=max_evals,
                             trials=trials)
    best_params = space_eval(space, best_space_params)
    return best_params, trials

def get_objective_function(estimator_to_optimize, cv, x_train, y_train):
    def objective(current_space):
        estimator = estimator_to_optimize.set_params(**current_space)
        scores = cross_val_score(estimator, x_train, y_train, cv=cv)
        print "Iteration %s: score %s with std %s for params %s" % (objective.current_iteration,
                                                                    scores.mean(),
                                                                    scores.std(),
                                                                    str(current_space))
        objective.current_iteration += 1
        return {'loss': 1 - scores.mean(),
                'status': STATUS_OK,
                'eval_time': time.time()}

    objective.current_iteration = 0
    return objective


In [82]:
get_best_params(model_pipeline, parameters_space, 5, x, y, 10)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(steps=[('random_forest', RandomForestRegressor(bootstrap=True, criterion='entropy', max_depth=9,
           max_features=None, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=133.0, n_jobs=4, oob_score=False,
           random_state=None, verbose=0, warm_start=False)), ('variance_threshold', VarianceThreshold(threshold=0.743458122638))]) does not.